In [1]:
from IPython.display import Markdown, display # for markdown text
import json # for json methods
import pprint # to print human readable dictionary
import pandas as pd # for visualizations
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer # for TF-IDF

In [2]:
def printmd(string):
    display(Markdown(string))

# FETCHING THE DATA

## All the topics in our database

In [3]:
topics = json.load(open('topics.txt'))
pprint.pprint(topics)

{'16': 'openhardware',
 '18': 'Data Science',
 '19': 'Big Data',
 '20': 'Artificial Intelligence',
 '21': 'Business Intelligence',
 '31': 'arduino',
 '32': 'raspberry pi',
 '33': '3d printer',
 '36': 'Deep Learning',
 '37': 'IoT',
 '38': '3d printing',
 '39': 'open hardware',
 '56': 'Wearable',
 '57': 'Sustainable finance',
 '59': 'Sustainable Finance',
 '60': 'Climate Finance',
 '61': 'Green Bonds',
 '62': 'Green Economy'}


## The parameters in our scenario
We are observing an audience, which is defined by two constraints: a topic and a location. Our example is the audience in Italy interested in the topic: Arduino.

In [4]:
TOPIC_ID=31 # topic = arduino
LOCATION = 'italy'
SIGNAL_STRENGTH = 0 # this value indicates the min number of influencers the retrieved audience members follow within the topic
LIMIT = 50 # number of audience members to consider
TESTING_SET_SIZE=10
HOW_MANY_TWEETS = 50 # amount of most recent tweets (including retweets) to be retrieved to consider in our recommendation engine
INCLUDE_RETWEETS = True

In [5]:
rated_audience_dict = json.load(open('rated_audience.txt'))
unrated_audience_dict = json.load(open('unrated_audience.txt'))

In [6]:
printmd("## An example Twitter profile with all the data fields at this point.")
printmd("### Topic: " + topics[str(TOPIC_ID)])
pprint.pprint(next (iter (rated_audience_dict.values())))

## An example Twitter profile with all the data fields at this point.

### Topic: arduino

{'description': 'Chief Innovation Officer at NTT DATA Italia , proud father of '
                'two beautiful girls, a professor, a technology evangelist, an '
                'holistic thinker and a gentleman.',
 'ground_truth_rating': 1.0,
 'hashtags': 'Milano CheTempoFa Milano CheTempoFa foi13 Cefriel fvw2013 '
             'fvw2013 fvw2013 storytelling Vajont Milano CheTempoFa '
             'StartupWeekend GrandC4Picasso makerfairerome MakerFaireRome '
             'GrandC4Picasso GrandC4Picasso',
 'influencers': '266400754 84094835 767285',
 'location': 'Milan, Italy',
 'screen_name': 'funkysurfer',
 'tweets': 'at Cascina Matiot Disorders of Con Edi Touch is easier than yours '
           'via Startup weekend 3 days at Milan Decoded Milan capital of the '
           'street October 2013 Rain Massima Minima The new science is an open '
           'narration thanks to This project was born via Milan handmade with '
           'Craft Camp of a Pisa ItCup A Roncade in Veneto via Oc

In [7]:
# Computes the TF-IDF values for the given corpus.
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 0, stop_words = 'english');
def get_tfidf(corpus):
    return tf.fit_transform(corpus.copy()).todense();

## Separate the data into different arrays

In [8]:
screen_names = [aud['screen_name'] for aud in rated_audience_dict.values()]
influencers_corpus = [aud['influencers'] for aud in rated_audience_dict.values()]
tweets_corpus = [aud['tweets'] for aud in rated_audience_dict.values()]
hashtags_corpus = [aud['hashtags'] for aud in rated_audience_dict.values()]
description_corpus = [aud['description'] for aud in rated_audience_dict.values()]
ground_truth_ratings = np.array([2*aud['ground_truth_rating'] for aud in rated_audience_dict.values()])
#print(tweets_corpus)

In [9]:
# INFLUENCER MATRICES
tfidf_influencer_matrix = np.array(get_tfidf(influencers_corpus)).T;
binary_influencer_matrix = tfidf_influencer_matrix.copy()
binary_influencer_matrix[binary_influencer_matrix>0]=1
# TWEET MATRICES
tfidf_tweets_matrix=np.array(get_tfidf(tweets_corpus)).T;
binary_tweets_matrix = tfidf_tweets_matrix.copy()
binary_tweets_matrix[binary_tweets_matrix>0]=1
# DESCRIPTION MATRICES
tfidf_description_matrix = np.array(get_tfidf(description_corpus)).T;
binary_description_matrix = tfidf_description_matrix.copy()
binary_description_matrix[binary_description_matrix>0]=1
# HASHTAGS MATRICES
tfidf_hashtags_matrix = np.array(get_tfidf(hashtags_corpus)).T;
binary_hashtags_matrix = tfidf_hashtags_matrix.copy()
binary_hashtags_matrix[binary_hashtags_matrix>0]=1
printmd("#### Matrix dimensions")
printmd("Influencer matrix: " + str(tfidf_influencer_matrix.shape))
printmd("Tweets matrix: " +str(tfidf_tweets_matrix.shape))
printmd("Description matrix: " +str(tfidf_description_matrix.shape))
printmd("Hashtags matrix: " +str(tfidf_hashtags_matrix.shape))

#### Matrix dimensions

Influencer matrix: (22, 50)

Tweets matrix: (4859, 50)

Description matrix: (308, 50)

Hashtags matrix: (1158, 50)

# METHODS FOR RECOMMENDATION

# Method A
## Logistic Regression using the similarities coming from user profiling (relaxed parameters)

In [10]:
from tfidf_vectorizer import TwitterAccountSimilarityFinder
import mord as md
from sklearn import datasets, linear_model

In [11]:
from scipy import sparse
def get_user_profile(tfidf_matrix, ratings):
    return np.multiply(tfidf_matrix.T,ratings[:, np.newaxis]).sum(axis=0).reshape(1,tfidf_matrix.shape[0])

In [12]:
def cos_sim(tfidf_matrix, user_profile):
    norm = np.linalg.norm(user_profile);
    return 1.0* tfidf_matrix.T.dot(user_profile.T)/ norm
    

In [15]:
SUBSAMPLING_COUNT = 20 # we will subsample this many times and take the average error for evaluation
rated_audience_dict_ids = list(rated_audience_dict.keys())

In [17]:
mean_square_errors = []
prints_enabled = False
for iteration in range(SUBSAMPLING_COUNT):
    predictions=[]
    TESTING_SET_IDS = np.random.choice(rated_audience_dict_ids, size=TESTING_SET_SIZE, replace=False)
    if (prints_enabled): print(TESTING_SET_IDS)
    ratings = np.array([0 if id in TESTING_SET_IDS else int(2*aud['ground_truth_rating']) for id, aud in rated_audience_dict.items()])
    if (prints_enabled): display(ratings)
        
    # FIND USER PROFILES
    influencer_user_profile = get_user_profile(tfidf_influencer_matrix, ratings)
    tweets_user_profile = get_user_profile(tfidf_tweets_matrix, ratings)
    description_user_profile = get_user_profile(tfidf_description_matrix, ratings)
    hashtags_user_profile = get_user_profile(tfidf_hashtags_matrix, ratings)

    # FIND THE COSINE SIMILARITIES
    # THESE WILL THEN BE USED AS FEATURES IN REGRESSION
    influencerSimilarities = cos_sim(tfidf_influencer_matrix, influencer_user_profile)
    tweetSimilarities = cos_sim(tfidf_tweets_matrix, tweets_user_profile)
    descriptionSimilarities = cos_sim(tfidf_description_matrix, description_user_profile)
    hashtagSimilarities = cos_sim(tfidf_hashtags_matrix, hashtags_user_profile)
    
    profile_count = len(rated_audience_dict)
    avgTweetSim = np.mean([e for e in tweetSimilarities if e!=0]) # if we cannot fetch tweets of a profile, we assign his tweets an average similarity score
    avgDescriptionSim = np.mean([e for e in descriptionSimilarities if e!=0]) # if the description of a profile is empty, we assign his descripion an average similarity score
    avgHashtagSim = np.mean([e for e in hashtagSimilarities if e!=0]) # if we cannot fetch tweets of a profile, we assign his hashtags an average similarity score
    
    profiles =[]
    training_indices=[]
    for i in range(profile_count):
        if tweetSimilarities[i]==0: tweetSimilarities[i]= avgTweetSim
        if descriptionSimilarities[i]==0: descriptionSimilarities[i]= avgDescriptionSim
        if hashtagSimilarities[i]==0: hashtagSimilarities[i] = avgHashtagSim
        if ratings[i]!=0: training_indices.append(i)
        profile = {
        'index': i,
        'screen_name':screen_names[i],
        'set': "testing" if ratings[i]==0 else "training",
        'ground_truth':ground_truth_ratings[i],
        'infSim': influencerSimilarities[i].item(0),
        'tweetSim': tweetSimilarities[i].item(0),
        'descSim': descriptionSimilarities[i].item(0),
        'hashtagSim': hashtagSimilarities[i].item(0),
        'score': 0
        }
        count=0
        for key in profile.keys():
            if 'Sim' in key: 
                profile['score']+=profile[key]
                count+=1
        profile['score']/=count*1.0
        profiles.append(profile)

    profiles=pd.DataFrame(profiles)
    if (prints_enabled):
        display(profiles[['screen_name','set','ground_truth','score','infSim','tweetSim','descSim','hashtagSim']].sort_values(by='score',ascending=False).round(2))
        printmd("### Naive approach\n Score is the average of similarities.")
        profiles.plot.scatter('score','ground_truth') # score is the average of the similarities
    
    # Combine the similarities and use them as features to feed to a logistic regressor.
    # uncomment to add the similarity into regression

    featureVectors = influencerSimilarities
    featureVectors = np.column_stack((featureVectors,tweetSimilarities))
    #featureVectors = np.column_stack((featureVectors, descriptionSimilarities))
    #featureVectors = np.column_stack((featureVectors, hashtagSimilarities))

    X = np.array([featureVectors[i] for i in training_indices])
    X_binary = X.copy()
    X_binary[X_binary>0]=1
    
    Y = np.array([ratings[i] for i in training_indices])
    Y_binary = [round(e/10) for e in Y]
    
    # Ordinal Regression
    #classifier = linear_model.LinearRegression()
    # Logistic Regression
    #classifier = md.LogisticIT() #Default parameters: alpha=1.0, verbose=0, maxiter=10000
    
    classifier = linear_model.LogisticRegression(C=1e5)
    #print(X)
    #print(Y)
    classifier.fit(X, Y)

    predictions = classifier.predict(featureVectors)
    profiles['predicted_rating']=predictions
    profiles["squared_error"]=(profiles["ground_truth"]-profiles["predicted_rating"])**2
    if (prints_enabled):
        display(profiles[['screen_name','set','ground_truth', 'predicted_rating','squared_error','score','infSim','tweetSim','descSim','hashtagSim']].sort_values(by='score',ascending=False).round(2))

    evaluation = pd.DataFrame()
    evaluation['mean_squared_error']=profiles.groupby(by='set')['squared_error'].mean()
    mean_square_error = {
        'training':evaluation.filter(like='training', axis=0)['mean_squared_error'].iloc[0],
        'testing':evaluation.filter(like='testing', axis=0)['mean_squared_error'].iloc[0],
        }
    mean_square_errors.append(mean_square_error)
    
Avg_MSE_training = np.mean([e['training'] for e in mean_square_errors])
Avg_MSE_testing = np.mean([e['testing'] for e in mean_square_errors])
print("Avg Training Mean Squared Error: " + str(Avg_MSE_training))
print("Avg Testing Mean Squared Error: " + str(Avg_MSE_testing))

Avg Training Mean Squared Error: 5.84125
Avg Testing Mean Squared Error: 13.575


# MATRIX METHODS

In [18]:
import math

In [19]:
# Matrix Factorization via multiplicative update rule
# Original Author: Ali Taylan Cemgil from Bogazici University
def nmf_kl_multiplicative(D, M, W, H, EPOCH=5000):
    MD = D.copy()
    MD[M==0] = 0
    for e in range(EPOCH):
        Xhat = W.dot(H)
        W=W*np.array(((MD/Xhat).dot(H.T)/np.dot(M, H.T)))
        Xhat = W.dot(H)
        H = H*np.array((W.T.dot(MD/Xhat)/np.dot(W.T, M)))
        #print(np.sum(np.abs(MD - M*Xhat))/np.sum(M))
    return W, H

In [20]:
# Regularized Matrix Factorization 
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.15):
    Q = Q.T
    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        if(i==0):
                            P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        else:         
                            P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        if(k==0):
                            Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
                        else:
                            Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

#         eR = np.dot(P,Q)
#         e = 0
#         for i in range(len(R)):
#             for j in range(len(R[i])):
#                 if R[i][j] > 0:
#                     e = e + pow(R[i][j] - np.dot(P[i,:],Q[:,j]), 2)
#                     for k in range(K):
#                         e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
#         if e < 0.001:
#             break
    return P, Q.T

In [21]:
tfidf_data_matrix =[]
tfidf_data_matrix = tfidf_influencer_matrix # first add influencers
tfidf_data_matrix = np.append(tfidf_data_matrix,tfidf_tweets_matrix,axis=0) # then tweets
tfidf_data_matrix = np.append(tfidf_data_matrix,tfidf_description_matrix,axis=0) # then descriptions
tfidf_data_matrix = np.append(tfidf_data_matrix,tfidf_hashtags_matrix,axis=0) # then hashtags

binary_data_matrix =[]
binary_data_matrix = binary_influencer_matrix # first add influencers
binary_data_matrix = np.append(binary_data_matrix,binary_tweets_matrix,axis=0) # then tweets
binary_data_matrix = np.append(binary_data_matrix,binary_description_matrix,axis=0) # then descriptions
binary_data_matrix = np.append(binary_data_matrix,binary_hashtags_matrix,axis=0) # then hashtags

#display(pd.DataFrame(binary_data_matrix))

# Method B 
## Non-negative Matrix Factorization (NMF) without the ratings row = Document Clustering

In [22]:
mean_square_errors = []
prints_enabled = False
#Rank
R = 10
# Data
Nr = tfidf_data_matrix.shape[0]
Nc = tfidf_data_matrix.shape[1]

for iteration in range(SUBSAMPLING_COUNT):
    predictions=[]
    TESTING_SET_IDS = np.random.choice(rated_audience_dict_ids, size=TESTING_SET_SIZE, replace=False)
    #if (prints_enabled): print(TESTING_SET_IDS)
    ratings = np.array([0 if id in TESTING_SET_IDS else int(2*aud['ground_truth_rating']) for id, aud in rated_audience_dict.items()])
    if (prints_enabled): 
        printmd("Ratings:")
        display(ratings)

    # Initialize W and H with random numbers
    W = np.random.rand(Nr, R)*100
    H = np.random.rand(R, Nc)*100

    Mask = np.ones_like(tfidf_data_matrix)
    Mask[np.isnan(tfidf_data_matrix)] = 0

    W,H = nmf_kl_multiplicative(tfidf_data_matrix, Mask, W, H, EPOCH=1)

    # Cluster numbers holds which cluster each user is assigned to
    cluster_numbers = np.argmax(H,axis=0)
    # Ratings each cluster is assigned to (initialized to zeros)
    cluster_ratings = np.zeros(R)

    for i in range(len(cluster_numbers)):
        cluster_no = cluster_numbers[i]
        cluster_ratings[cluster_no]+=1.0*ratings[i] 

    for i in range(R):
        cluster_size = float(len(cluster_numbers[cluster_numbers==i]))
        if cluster_size!=0:
            cluster_ratings[i]/=cluster_size
   
    predictions = [np.around(cluster_ratings[cluster_no]) for cluster_no in cluster_numbers]    
    # Found the cluster ratings
    cluster_ratings = [{'cluster_no':i, 'rating':cluster_ratings[i]} for i in range(R)]
    cluster_ratings = pd.DataFrame(cluster_ratings).round(2)
    if (prints_enabled): 
        printmd("Cluster numbers:")
        display(cluster_numbers)
        printmd("Cluster ratings:")
        display(cluster_ratings)
    
    results = pd.DataFrame()
    results['set'] = ['training' if ratings[i]!=0 else 'testing' for i in range(len(ratings)) ]
    results['ground_truth']=ground_truth_ratings
    results['predicted_rating']=predictions
    results["squared_error"]=(results["ground_truth"]-results["predicted_rating"])**2
    
    if (prints_enabled):
        display(results[['set','ground_truth','predicted_rating','squared_error']].round(2))

    evaluation = pd.DataFrame()
    evaluation['mean_squared_error']=results.groupby(by='set')['squared_error'].mean()
    mean_square_error = {
        'training':evaluation.filter(like='training', axis=0)['mean_squared_error'].iloc[0],
        'testing':evaluation.filter(like='testing', axis=0)['mean_squared_error'].iloc[0],
        }
    mean_square_errors.append(mean_square_error)
    
Avg_MSE_training = np.mean([e['training'] for e in mean_square_errors])
Avg_MSE_testing = np.mean([e['testing'] for e in mean_square_errors])
print("Avg Training Mean Squared Error: " + str(Avg_MSE_training))
print("Avg Testing Mean Squared Error: " + str(Avg_MSE_testing))
    

Avg Training Mean Squared Error: 6.87875
Avg Testing Mean Squared Error: 9.5


# Method D 
## Non-negative Matrix Factorization (NMF) with the ratings row

In [23]:
mean_square_errors = []
prints_enabled = False
#Rank
R = 10
# Data
Nr = tfidf_data_matrix.shape[0]+1
Nc = tfidf_data_matrix.shape[1]

for iteration in range(SUBSAMPLING_COUNT):
    predictions=[]
    TESTING_SET_IDS = np.random.choice(rated_audience_dict_ids, size=TESTING_SET_SIZE, replace=False)
    #if (prints_enabled): print(TESTING_SET_IDS)
    ratings = np.array([0 if id in TESTING_SET_IDS else int(2*aud['ground_truth_rating']) for id, aud in rated_audience_dict.items()])
    if (prints_enabled): 
        printmd("Ratings:")
        display(ratings)

    tfidf_data_matrix_with_ratings = ratings.reshape(1,len(ratings))
    tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,tfidf_data_matrix,axis=0)

    # Initialize W and H with random numbers
    W = np.random.rand(Nr, R)*100
    H = np.random.rand(R, Nc)*100

    Mask = np.ones_like(tfidf_data_matrix_with_ratings)
    Mask[np.isnan(tfidf_data_matrix_with_ratings)] = 0

    W,H = nmf_kl_multiplicative(tfidf_data_matrix_with_ratings, Mask, W, H, EPOCH=1)
    Xhat = W.dot(H)
    predictions = Xhat[0]
    predictions = np.around(10.0*predictions/max(predictions))
    
    results = pd.DataFrame()
    results['set'] = ['training' if ratings[i]!=0 else 'testing' for i in range(len(ratings)) ]
    results['ground_truth']=ground_truth_ratings
    results['predicted_rating']=predictions
    results["squared_error"]=(results["ground_truth"]-results["predicted_rating"])**2
    
    if (prints_enabled):
        display(results[['set','ground_truth','predicted_rating','squared_error']].round(2))

    evaluation = pd.DataFrame()
    evaluation['mean_squared_error']=results.groupby(by='set')['squared_error'].mean()
    mean_square_error = {
        'training':evaluation.filter(like='training', axis=0)['mean_squared_error'].iloc[0],
        'testing':evaluation.filter(like='testing', axis=0)['mean_squared_error'].iloc[0],
        }
    mean_square_errors.append(mean_square_error)
    
Avg_MSE_training = np.mean([e['training'] for e in mean_square_errors])
Avg_MSE_testing = np.mean([e['testing'] for e in mean_square_errors])
print("Avg Training Mean Squared Error: " + str(Avg_MSE_training))
print("Avg Testing Mean Squared Error: " + str(Avg_MSE_testing))

Avg Training Mean Squared Error: 7.8525
Avg Testing Mean Squared Error: 9.49


In [24]:
mean_square_errors = []
prints_enabled = False
#Rank
R = 10
# Data
Nr = tfidf_data_matrix.shape[0]+1
Nc = tfidf_data_matrix.shape[1]
for T in np.arange(1,10,0.1):
    for iteration in range(1):
        predictions=[]
        TESTING_SET_IDS = np.random.choice(rated_audience_dict_ids, size=TESTING_SET_SIZE, replace=False)
        #if (prints_enabled): print(TESTING_SET_IDS)
        ratings = np.array([0 if id in TESTING_SET_IDS else int(2*aud['ground_truth_rating']) for id, aud in rated_audience_dict.items()])
        if (prints_enabled): 
            printmd("Ratings:")
            display(ratings)

        tfidf_data_matrix_with_ratings = ratings.reshape(1,len(ratings))
        tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,tfidf_data_matrix,axis=0)

        # Initialize W and H with random numbers
        W = np.random.rand(Nr, R)*100
        H = np.random.rand(R, Nc)*100

        Mask = np.ones_like(tfidf_data_matrix_with_ratings)
        Mask[np.isnan(tfidf_data_matrix_with_ratings)] = 0

        W,H = nmf_kl_multiplicative(tfidf_data_matrix_with_ratings, Mask, W, H, EPOCH=1)
        Xhat = W.dot(H)
        predictions = Xhat[0]
        predictions = np.around(10.0*predictions/max(predictions))


        results = pd.DataFrame()
        results['set'] = ['training' if ratings[i]!=0 else 'testing' for i in range(len(ratings)) ]
        results['ground_truth']=[0 if r<=T else 1 for r in ground_truth_ratings] # binary ground truth
        results['predicted_rating']=[0 if r<=T else 1 for r in predictions] # binary predictions
        results["squared_error"]=(results["ground_truth"]-results["predicted_rating"])**2

        if (prints_enabled):
            display(results[['set','ground_truth','predicted_rating','squared_error']].round(2))

        evaluation = pd.DataFrame()
        evaluation['mean_squared_error']=results.groupby(by='set')['squared_error'].mean()
        mean_square_error = {
            'training':evaluation.filter(like='training', axis=0)['mean_squared_error'].iloc[0],
            'testing':evaluation.filter(like='testing', axis=0)['mean_squared_error'].iloc[0],
            }
        mean_square_errors.append(mean_square_error)
        
    #Avg_MSE_training = np.mean([e['training'] for e in mean_square_errors])
    Avg_MSE_testing = np.mean([e['testing'] for e in mean_square_errors])
    #print("Avg Training Mean Squared Error: " + str(Avg_MSE_training))
    print("Avg Testing Mean Squared Error: " + str(Avg_MSE_testing.round(2)) + " Threshold: " + str(T))
    


Avg Testing Mean Squared Error: 0.2 Threshold: 1.0
Avg Testing Mean Squared Error: 0.15 Threshold: 1.1
Avg Testing Mean Squared Error: 0.1 Threshold: 1.2
Avg Testing Mean Squared Error: 0.1 Threshold: 1.3
Avg Testing Mean Squared Error: 0.08 Threshold: 1.4
Avg Testing Mean Squared Error: 0.07 Threshold: 1.5
Avg Testing Mean Squared Error: 0.07 Threshold: 1.6
Avg Testing Mean Squared Error: 0.06 Threshold: 1.7
Avg Testing Mean Squared Error: 0.06 Threshold: 1.8
Avg Testing Mean Squared Error: 0.07 Threshold: 1.9
Avg Testing Mean Squared Error: 0.09 Threshold: 2.0
Avg Testing Mean Squared Error: 0.1 Threshold: 2.1
Avg Testing Mean Squared Error: 0.12 Threshold: 2.2
Avg Testing Mean Squared Error: 0.13 Threshold: 2.3
Avg Testing Mean Squared Error: 0.15 Threshold: 2.4
Avg Testing Mean Squared Error: 0.16 Threshold: 2.5
Avg Testing Mean Squared Error: 0.15 Threshold: 2.6
Avg Testing Mean Squared Error: 0.17 Threshold: 2.7
Avg Testing Mean Squared Error: 0.17 Threshold: 2.8
Avg Testing Mean

In [25]:
mean_square_errors = []
prints_enabled = False
K = 16
for iteration in range(SUBSAMPLING_COUNT):
    predictions=[]
    TESTING_SET_IDS = np.random.choice(rated_audience_dict_ids, size=TESTING_SET_SIZE, replace=False)
    #if (prints_enabled): print(TESTING_SET_IDS)
    ratings = np.array([0 if id in TESTING_SET_IDS else int(2*aud['ground_truth_rating']) for id, aud in rated_audience_dict.items()])
    if (prints_enabled): 
        printmd("Ratings:")
        display(ratings)

    tfidf_data_matrix_with_ratings = ratings.reshape(1,len(ratings))
    tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,tfidf_influencer_matrix,axis=0)

    # Initialize W and H with random numbers    
    R = tfidf_data_matrix_with_ratings
    N= len(R)
    M = len(R[0])
    steps = 2000

    P = np.random.rand(N,K)
    Q = np.random.rand(M,K)

    nP, nQ = matrix_factorization(R, P, Q, K,steps)
    nR = np.dot(nP, nQ.T)

    predictions =np.round(nR[0].reshape(1,len(nR[0])))[0]
    print(predictions)
    
    results = pd.DataFrame()
    results['set'] = ['training' if ratings[i]!=0 else 'testing' for i in range(len(ratings)) ]
    results['ground_truth']=ground_truth_ratings
    results['predicted_rating']=predictions
    results["squared_error"]=(results["ground_truth"]-results["predicted_rating"])**2
    
    if (prints_enabled):
        display(results[['set','ground_truth','predicted_rating','squared_error']].round(2))

    evaluation = pd.DataFrame()
    evaluation['mean_squared_error']=results.groupby(by='set')['squared_error'].mean()
    mean_square_error = {
        'training':evaluation.filter(like='training', axis=0)['mean_squared_error'].iloc[0],
        'testing':evaluation.filter(like='testing', axis=0)['mean_squared_error'].iloc[0],
        }
    mean_square_errors.append(mean_square_error)
    
Avg_MSE_training = np.mean([e['training'] for e in mean_square_errors])
Avg_MSE_testing = np.mean([e['testing'] for e in mean_square_errors])
print("Avg Training Mean Squared Error: " + str(Avg_MSE_training))
print("Avg Testing Mean Squared Error: " + str(Avg_MSE_testing))

[  2.   4.   6.   4.   5.  10.  10.   9.   3.   5.   6.   8.   7.   3.   2.
   2.   6.   6.   2.   5.   8.   2.   1.   4.   8.   9.   5.   6.   4.   5.
   2.   7.   5.   6.   2.   4.   5.   4.   3.   8.   4.   2.   3.   3.   2.
   6.   5.  10.   8.   5.]
[  2.   5.   6.   4.   5.  10.  10.   9.   3.   5.   4.   8.   7.   3.   2.
   2.   5.   2.   4.   5.   8.   2.   1.   4.   7.   3.   5.   4.   4.   4.
   2.   6.   4.   6.   2.   7.   1.   4.   2.   8.   5.   2.   4.   3.   2.
   6.   5.   4.   8.   5.]
[  2.   1.   6.   4.   5.   2.  10.   9.   3.   5.   6.   3.   7.   3.   2.
   4.   5.   2.   2.   5.   8.   2.   3.   6.   7.   9.   5.   9.   4.   3.
   2.   7.   5.   6.   4.   7.   4.   4.   2.   8.   4.   2.   3.   2.   2.
   6.   5.  10.   8.   5.]
[  2.   1.   5.   4.   5.  10.  10.   9.   3.   5.   6.   3.   5.   3.   2.
   2.   5.   2.   4.   5.   8.   2.   1.   4.   8.   9.   5.   5.   4.   3.
   2.   7.   5.   6.   2.   7.   3.   4.   2.   5.   4.   2.   4.   3.   2.
   6.  

In [26]:
mean_square_errors = []
prints_enabled = False
K = 16
for iteration in range(SUBSAMPLING_COUNT):
    print(str(iteration))
    predictions=[]
    TESTING_SET_IDS = np.random.choice(rated_audience_dict_ids, size=TESTING_SET_SIZE, replace=False)
    #if (prints_enabled): print(TESTING_SET_IDS)
    ratings = np.array([0 if id in TESTING_SET_IDS else int(2*aud['ground_truth_rating']) for id, aud in rated_audience_dict.items()])
    if (prints_enabled): 
        printmd("Ratings:")
        display(ratings)
        
    # FIND USER PROFILES
    influencer_user_profile = get_user_profile(tfidf_influencer_matrix, ratings)
    tweets_user_profile = get_user_profile(tfidf_tweets_matrix, ratings)
    description_user_profile = get_user_profile(tfidf_description_matrix, ratings)
    hashtags_user_profile = get_user_profile(tfidf_hashtags_matrix, ratings)

    # FIND THE COSINE SIMILARITIES
    # THESE WILL THEN BE USED AS FEATURES IN REGRESSION
    influencerSimilarities = cos_sim(tfidf_influencer_matrix, influencer_user_profile)
    tweetSimilarities = cos_sim(tfidf_tweets_matrix, tweets_user_profile)
    descriptionSimilarities = cos_sim(tfidf_description_matrix, description_user_profile)
    hashtagSimilarities = cos_sim(tfidf_hashtags_matrix, hashtags_user_profile)
    
    avgTweetSim = np.mean([e for e in tweetSimilarities if e!=0]) # if we cannot fetch tweets of a profile, we assign his tweets an average similarity score
    avgDescriptionSim = np.mean([e for e in descriptionSimilarities if e!=0]) # if the description of a profile is empty, we assign his descripion an average similarity score
    avgHashtagSim = np.mean([e for e in hashtagSimilarities if e!=0]) # if we cannot fetch tweets of a profile, we assign his hashtags an average similarity score
    tweetSimilarities[tweetSimilarities==0]=avgTweetSim
    descriptionSimilarities[descriptionSimilarities==0]=avgDescriptionSim
    hashtagSimilarities[hashtagSimilarities==0]=avgHashtagSim

    
    tfidf_data_matrix_with_ratings = ratings.reshape(1,len(ratings))
    #tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,influencerSimilarities.T,axis=0)
    tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,tweetSimilarities.T,axis=0)
    tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,descriptionSimilarities.T,axis=0)
    tfidf_data_matrix_with_ratings = np.append(tfidf_data_matrix_with_ratings,hashtagSimilarities.T,axis=0)
    display(pd.DataFrame(tfidf_data_matrix_with_ratings))

    # Initialize W and H with random numbers    
    R = tfidf_data_matrix_with_ratings
    N= len(R)
    M = len(R[0])
    steps = 2000

    P = np.random.rand(N,K)
    Q = np.random.rand(M,K)

    nP, nQ = matrix_factorization(R, P, Q, K,steps)
    nR = np.dot(nP, nQ.T)

    predictions =np.round(nR[0].reshape(1,len(nR[0])))[0]
    #print(predictions)
    
    results = pd.DataFrame()
    results['set'] = ['training' if ratings[i]!=0 else 'testing' for i in range(len(ratings)) ]
    results['ground_truth']=ground_truth_ratings
    results['predicted_rating']=predictions
    results["squared_error"]=(results["ground_truth"]-results["predicted_rating"])**2
    
    #if (prints_enabled):
    #display(results[['set','ground_truth','predicted_rating','squared_error']].round(2))

    evaluation = pd.DataFrame()
    evaluation['mean_squared_error']=results.groupby(by='set')['squared_error'].mean()
    mean_square_error = {
        'training':evaluation.filter(like='training', axis=0)['mean_squared_error'].iloc[0],
        'testing':evaluation.filter(like='testing', axis=0)['mean_squared_error'].iloc[0],
        }
    mean_square_errors.append(mean_square_error)
    
Avg_MSE_training = np.mean([e['training'] for e in mean_square_errors])
Avg_MSE_testing = np.mean([e['testing'] for e in mean_square_errors])
print("Avg Training Mean Squared Error: " + str(Avg_MSE_training))
print("Avg Testing Mean Squared Error: " + str(Avg_MSE_testing))

0


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,2.000000,1.000000,6.000000,0.000000,0.000000,10.000000,10.000000,10.000000,0.000000,5.000000,...,4.000000,2.000000,0.000000,3.000000,2.000000,6.000000,5.000000,10.000000,8.000000,0.000000
1,0.202226,0.229539,0.374722,0.094930,0.138711,0.271734,0.335465,0.431838,0.147519,0.238388,...,0.210672,0.314559,0.125145,0.154266,0.284111,0.213084,0.241173,0.387186,0.302048,0.088992
2,0.120826,0.165675,0.201757,0.116149,0.101942,0.361705,0.284369,0.300860,0.168906,0.217022,...,0.173389,0.055337,0.021280,0.168531,0.055337,0.296727,0.219106,0.306070,0.290601,0.085894
3,0.060263,0.031697,0.180316,0.006864,0.005171,0.286247,0.287788,0.303995,0.016504,0.232666,...,0.112442,0.056221,0.058934,0.084331,0.067481,0.171794,0.140552,0.281105,0.237704,0.031782


1


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,2.000000,1.000000,6.000000,4.000000,0.000000,10.000000,10.000000,10.000000,3.000000,0.000000,...,4.000000,2.000000,4.000000,3.000000,2.000000,6.000000,5.000000,10.000000,8.000000,0.000000
1,0.195031,0.230967,0.378900,0.173357,0.138385,0.264420,0.334211,0.442817,0.203834,0.143525,...,0.208781,0.321067,0.197413,0.156327,0.288590,0.219996,0.223308,0.380697,0.287009,0.095684
2,0.121916,0.177790,0.181131,0.226093,0.075360,0.342794,0.282109,0.276120,0.252589,0.106894,...,0.118502,0.051345,0.126700,0.198215,0.051345,0.260550,0.214043,0.351355,0.254474,0.085638
3,0.057860,0.034755,0.193159,0.107957,0.015761,0.278738,0.275109,0.307752,0.096185,0.082065,...,0.110459,0.053978,0.216695,0.080968,0.056183,0.164941,0.134946,0.271917,0.222788,0.020314


2


...

18


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,2.000000,1.000000,0.000000,4.000000,5.000000,10.000000,10.000000,10.000000,3.000000,5.000000,...,4.000000,2.000000,0.000000,3.000000,2.000000,6.000000,0.000000,10.000000,8.000000,5.000000
1,0.209083,0.223751,0.256652,0.184435,0.243060,0.275279,0.341389,0.396991,0.205257,0.246506,...,0.221237,0.270866,0.130679,0.163123,0.246536,0.236173,0.141925,0.378014,0.298500,0.204902
2,0.127129,0.172240,0.047557,0.217312,0.217118,0.328041,0.293272,0.278780,0.231179,0.226095,...,0.162643,0.053377,0.050831,0.179973,0.053377,0.265820,0.094334,0.340690,0.262281,0.205208
3,0.059372,0.031228,0.017987,0.117541,0.148510,0.280957,0.283780,0.316775,0.095782,0.227218,...,0.114702,0.055389,0.099319,0.083084,0.057924,0.169252,0.128355,0.276946,0.234983,0.162819


19


,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,2.000000,1.000000,0.000000,4.000000,5.000000,10.000000,10.000000,10.000000,3.000000,5.000000,...,4.000000,2.000000,4.000000,3.000000,2.000000,6.000000,5.000000,10.000000,8.000000,5.000000
1,0.209421,0.227521,0.274866,0.182511,0.245339,0.275254,0.339957,0.424855,0.197327,0.246546,...,0.217262,0.304199,0.209925,0.156705,0.268943,0.227345,0.228526,0.390019,0.307411,0.190914
2,0.125969,0.169464,0.036803,0.217053,0.208195,0.333658,0.303831,0.292932,0.271252,0.220504,...,0.175191,0.056087,0.146856,0.195992,0.056087,0.254204,0.234577,0.351915,0.275595,0.223676
3,0.061330,0.036840,0.028214,0.121418,0.152407,0.290223,0.294020,0.314562,0.101927,0.229206,...,0.115832,0.057216,0.175579,0.085824,0.066988,0.174834,0.143040,0.288227,0.239775,0.165896


Avg Training Mean Squared Error: 0.0
Avg Testing Mean Squared Error: 7.34
